In [20]:
import utils.dataloading as dlt
import utils.model_utils as mu
import utils.dl_utils as dl
import torch.nn as nn
from astropy.io.fits import getheader
from astropy.io import fits
import numpy as np
import torch
from tqdm import tqdm
import torchio as tio
from itertools import starmap

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
input = torch.randn(1, 1, 128, 128, 128)
freq = torch.randn(1, 2)
path = '/media/storage/big_cube/dirty_cube_0.fits'
segmenter = dlt.get_base_segmenter(path, (128, 128, 128))


In [ ]:
out = segmenter(input, freq)

In [ ]:
#import utils.dl_utils as dl 

path = '/media/storage/big_cube/dirty_cube_0.fits'
n_parallel = 1
i_job = 0
max_batch_size = 32
header = fits.getheader(path)
model_input_dim = np.array([128, 128, 128])
cnn_padding = np.array([8, 8, 8])
desired_dim = (model_input_dim - 2 * cnn_padding)
print(desired_dim)
segmenter = dlt.get_base_segmenter(path, (128, 128, 128))
device = torch.device('cuda')
evaluator = dlt.EvaluationTraverser(segmenter, path, model_input_dim, cnn_padding,
                                     desired_dim, max_batch_size, n_parallel=1, i_job=0)
df = evaluator.traverse(output_path='/media/storage/big_cube/processed.fits')

In [21]:
path = '/media/storage/big_cube/dirty_cube_0.fits'

validation_transforms = tio.Compose([
                        tio.RescaleIntensity((0, 1))])
dataset, aggregator = dl.get_big_cube_dataloaders(path, None, validation_transforms, (256, 256, 128), (8, 8, 8),
                                                   32, 256, 256,  10, False, True)

In [23]:

device = torch.device('cuda')
model = nn.Identity().to(device).eval()
with torch.no_grad():
    for patches_batch in tqdm(dataset, total=len(dataset)):
        print
        input_tensor = patches_batch['dirty'][tio.DATA].to(device)
        print(input_tensor.shape)
        locations = patches_batch[tio.LOCATION].to(device)
        output_tensor = model(input_tensor)
        aggregator.add_batch(output_tensor, locations)
output_tensor = aggregator.get_output_tensor()
print(output_tensor.shape)

  0%|          | 0/24 [00:00<?, ?it/s]

torch.Size([32, 1, 256, 256, 128])


  4%|▍         | 1/24 [00:15<05:58, 15.60s/it]

torch.Size([32, 1, 256, 256, 128])


  8%|▊         | 2/24 [00:16<02:32,  6.92s/it]

...

torch.Size([32, 1, 256, 256, 128])


 96%|█████████▌| 23/24 [00:38<00:00,  1.59it/s]

torch.Size([32, 1, 256, 256, 128])


100%|██████████| 24/24 [00:41<00:00,  1.75s/it]

torch.Size([1, 1280, 1880, 1880])
